In [4]:
from dotenv import load_dotenv

load_dotenv()


True

In [22]:
from pydantic import BaseModel
from textwrap import dedent
from openai import OpenAI

client = OpenAI()
MODEL = "gpt-4o-mini"

integral_solver_prompt = """
    You are a calculus expert capable of solving indefinite integrals
    step-by-step. Given an integral problem, provide a detailed solution
    including each integration step and the final answer. For each step, provide
    the mathematical expression and an explanation of the rule or technique
    used.
"""


class IntegralSolution(BaseModel):
    class Step(BaseModel):
        explanation: str
        """A brief explanation of the step. Please use LaTeX for the
        mathematical expressions. use $..$ to enclose
        inline expressions and $$..$$ to enclose block expressions."""
        expression: str
        """
        A mathematical step in LaTeX.
        e.g., "$$ \int_0^1{2x + 3} dx = \int_0^1{2x} dx + \int_0^1{3} dx $$"
        """

    steps: list[Step]
    final_answer: str
    """
    e.g., "$$x^2 + 3x + C$$"
    """


def solve_integral(integral_problem: str) -> IntegralSolution:
    """
    Solves an indefinite integral using an LLM and provides step-by-step reasoning.

    Parameters
    ----------
    integral_problem : str
        The integral problem to solve (e.g., "∫(2x + 3) dx").

    Returns
    -------
    IntegralSolution | None
        A Pydantic object containing the steps and final answer, or None if parsing fails.
    """
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(integral_solver_prompt)},
            {
                "role": "user",
                "content": f"Solve the following integral: {integral_problem}",
            },
        ],
        response_format=IntegralSolution,
    )
    return completion.choices[0].message.parsed

In [23]:
output = solve_integral("int[x^2 * sin(2x)dx] from 0 to 1")
from IPython.display import display, Markdown

for step in output.steps:
    display(Markdown(step.explanation))
    display(Markdown(step.expression))
    print()
display(Markdown(output.final_answer))

We will use integration by parts to solve the integral. The formula for integration by parts is \( \int u \, dv = uv - \int v \, du \). We choose \( u = x^2 \) and \( dv = \sin(2x) \, dx \).

u = x^2, \, dv = \sin(2x) \, dx.

Next, we compute \( du \) and \( v \). The derivative of \( u \) gives us \( du = 2x \, dx \). To find \( v \), we integrate \( dv \): \( v = -\frac{1}{2} \cos(2x) \).

du = 2x \, dx, \, v = -\frac{1}{2} \cos(2x).

Now we apply the integration by parts formula: \( \int x^2 \sin(2x) \, dx = uv - \int v \, du \). Therefore, we have: \( -\frac{1}{2} x^2 \cos(2x) \Big|_0^1 + \frac{1}{2} \int 2x \cos(2x) \, dx \).

...

Now assemblng everything, we have: Total integral = \( -\frac{1}{2} \cos(2) + \left( \frac{1}{2} \sin(2) - \frac{1}{4} \cos(2) \right) \).

= -\frac{1}{2} \cos(2) + \frac{1}{2} \sin(2) - \frac{1}{4} \cos(2) = \frac{1}{2} \sin(2) - \frac{3}{4} \cos(2).

Finally, we find the definite integral from 0 to 1, which gives us the answer as a function of sine and cosine values at x=2.

Final result = \frac{1}{2} \sin(2) - \frac{3}{4} \cos(2).

\frac{1}{2} \sin(2) - \frac{3}{4} \cos(2)

In [33]:
from pydantic import BaseModel, Field
from textwrap import dedent
from openai import OpenAI

client = OpenAI()
MODEL = "gpt-4o-mini"

integral_solver_prompt = """
    You are a calculus expert capable of solving indefinite integrals
    step-by-step. Given an integral problem, provide a detailed solution
    including each integration step and the final answer. For each step, provide
    the mathematical expression and an explanation of the rule or technique
    used.
"""


class IntegralSolution(BaseModel):
    class Step(BaseModel):
        explanation: str = Field(
            description="""
            A brief explanation of the step. Please use LaTeX for the
            mathematical expressions. use $..$ to enclose
            inline expressions and $$..$$ to enclose block expressions."""
        )
        expression: str = Field(
            description="""
            A mathematical step in LaTeX.
            e.g., "$$ \int_0^1{2x + 3} dx = \int_0^1{2x} dx + \int_0^1{3} dx $$"
        """
        )

    steps: list[Step]
    final_answer: str = Field(
        description="""
        e.g., "$$x^2 + 3x + C$$"
        """
    )


def solve_integral(integral_problem: str) -> IntegralSolution:
    """
    Solves an indefinite integral using an LLM and provides step-by-step reasoning.

    Parameters
    ----------
    integral_problem : str
        The integral problem to solve (e.g., "∫(2x + 3) dx").

    Returns
    -------
    IntegralSolution | None
        A Pydantic object containing the steps and final answer, or None if parsing fails.
    """
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(integral_solver_prompt)},
            {
                "role": "user",
                "content": f"Solve the following integral: {integral_problem}",
            },
        ],
        response_format=IntegralSolution,
    )
    return completion.choices[0].message.parsed

In [43]:
output = solve_integral("int[BesselJ(0,x)^2dx] from 0 to 1")

In [44]:
output

IntegralSolution(steps=[Step(explanation='We will use the property of the Bessel function, specifically the orthogonality relation of Bessel functions. One useful integral involving Bessel functions is $$ \\int_0^1 x J_0^2(x) \\, dx = \\frac{1}{2} \\int_0^1 x J_0^2(x) dx $$, where the relationship holds in the case of Bessel functions of the first kind. We first express our integral accordingly.', expression='$$ I = \\int_0^1 J_0^2(x) \\, dx $$')], final_answer='$$ \\frac{1}{4} $$')

In [42]:
for step in output.steps:
    print(f"> {step.explanation}")
    # print(f"> {step.expression}")
    print(">")
print(f"> {output.final_answer}")

> We need to solve the integral $$\int_0^1 x^2 \sin(2x) \, dx$$. To do this, we will use integration by parts, which is suitable for products of functions like $x^2$ and $\sin(2x)$. The formula for integration by parts is given by $$\int u \, dv = uv - \int v \, du.$$
>
> We choose our $u$ and $dv$ for integration by parts. Let $$u = x^2$$, then $$du = 2x \, dx$$. For $$dv = \sin(2x) \, dx$$, we integrate to find $$v$$. The integral of $\sin(2x)$ is $$v = -\frac{1}{2} \cos(2x)$$.
>
> Now we apply the integration by parts formula that was stated earlier. We will calculate $$uv$$ at the bounds of the integral and substitute it back into our equation.
>
> Evaluating the first part, we find the boundary values: when $x=1$, $$-\frac{1}{2} \cdot 1^2 \cdot \cos(2) = -\frac{1}{2} \cos(2)$$ and when $x=0$, it becomes $0$. Therefore, the evaluation gives us: $$-\frac{1}{2} \cos(2) - 0 = -\frac{1}{2} \cos(2).$$
>
> Now we need to compute the remaining integral: $$\int_0^1 x \cos(2x) \, dx$$. We a

In [38]:
from IPython.display import display, Markdown

for step in output.steps:
    display(Markdown(step.expression))
    # display(Markdown(step.expression))
    print()
display(Markdown(output.final_answer))

$$I = \int_0^1 x^2 \sin(2x) \, dx$$

$$u = x^2, \, du = 2x \, dx, \, dv = \sin(2x) \, dx, \, v = -\frac{1}{2} \cos(2x)$$

$$I = \left[ -\frac{1}{2} x^2 \cos(2x) \right]_0^1 - \int_0^1 -\frac{1}{2} \cos(2x) (2x) \, dx$$

$$\left[ -\frac{1}{2} x^2 \cos(2x) \right]_0^1 = -\frac{1}{2} \cos(2) - 0$$

$$\int_0^1 x \cos(2x) \, dx = \left[ \frac{1}{2} x \sin(2x) \right]_0^1 - \int_0^1 \frac{1}{2} \sin(2x) \, dx$$

$$\left[ \frac{1}{2} x \sin(2x) \right]_0^1 = \frac{1}{2} \sin(2) - 0 = \frac{1}{2} \sin(2)$$

$$-\frac{1}{2} \int_0^1 \sin(2x) \, dx = -\frac{1}{2} \left[-\frac{1}{2} \cos(2x) \right]_0^1 = -\frac{1}{4}[\cos(2) - 1]$$

$$I = -\frac{3}{4} \cos(2) + \frac{1}{2} \sin(2) + \frac{1}{4}$$

$$I = -\frac{3}{4} \cos(2) + \frac{1}{2} \sin(2) + \frac{1}{4}$$

$$I = -\frac{3}{4} \cos(2) + \frac{1}{2} \sin(2) + \frac{1}{4}$$